In [1]:
from _config import PKL_PROCESSED_STEP1_DTU_SOLAR_STATION
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Hyperparameters
INPUT_SEQ_LEN = 60   # past 60 minutes
OUTPUT_SEQ_LEN = 60  # predict next 60 minutes at 1-min resolution
BATCH_SIZE = 64
HIDDEN_DIM = 64
NUM_LAYERS = 2
FC_UNITS = 128
LEARNING_RATE = 1e-3
NUM_EPOCHS = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SolarIrradianceDataset(Dataset):
    def __init__(self, data, input_len=INPUT_SEQ_LEN, output_len=OUTPUT_SEQ_LEN):
        """
        data: pandas DataFrame with columns ['DNI', 'DHI']
        """
        self.values = data[['DNI', 'DHI']].values.astype(np.float32)
        self.input_len = input_len
        self.output_len = output_len
        self.indices = list(range(len(self.values) - input_len - output_len + 1))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        start = self.indices[idx]
        x = self.values[start : start + self.input_len]            # (input_len, 2)
        y = self.values[start + self.input_len : start + self.input_len + self.output_len]  # (output_len, 2)
        return torch.from_numpy(x), torch.from_numpy(y)

class DNI_DHI_LSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, output_len=OUTPUT_SEQ_LEN):
        super(DNI_DHI_LSTM, self).__init__()
        # two parallel LSTMs for each series
        self.lstm_dni = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                                num_layers=num_layers, batch_first=True)
        self.lstm_dhi = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                                num_layers=num_layers, batch_first=True)
        # combine hidden representations and map to full horizon of both outputs
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, FC_UNITS),
            nn.ReLU(),
            nn.Linear(FC_UNITS, output_len * 2)
        )

    def forward(self, x):
        # x: (batch, input_len, 2)
        x_dni = x[:, :, 0].unsqueeze(-1)  # (batch, input_len, 1)
        x_dhi = x[:, :, 1].unsqueeze(-1)

        _, (h_n_dni, _) = self.lstm_dni(x_dni)
        _, (h_n_dhi, _) = self.lstm_dhi(x_dhi)
        # take hidden from last layer
        latent_dni = h_n_dni[-1]  # (batch, hidden_dim)
        latent_dhi = h_n_dhi[-1]
        latent = torch.cat([latent_dni, latent_dhi], dim=1)  # (batch, hidden_dim*2)

        out = self.fc(latent)  # (batch, output_len*2)
        # reshape to (batch, output_len, 2)
        return out.view(-1, OUTPUT_SEQ_LEN, 2)


def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for x_batch, y_batch in loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x_batch.size(0)
    return total_loss / len(loader.dataset)


def eval_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            preds = model(x_batch)
            total_loss += criterion(preds, y_batch).item() * x_batch.size(0)
    return total_loss / len(loader.dataset)


def main():
    # Load your observed data (timestamp, DNI, DHI)
    df = pd.read_pickle(PKL_PROCESSED_STEP1_DTU_SOLAR_STATION)

    # Normalize between 0 and 1
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    df[['DNI', 'DHI']] = scaler.fit_transform(df[['DNI', 'DHI']])

    # Split train/test
    split = int(len(df)*0.8)
    train_df, test_df = df[:split], df[split:]

    # Datasets and loaders
    train_ds = SolarIrradianceDataset(train_df)
    test_ds  = SolarIrradianceDataset(test_df)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE)

    # Model, loss, optimizer
    model = DNI_DHI_LSTM().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Training loop
    for epoch in range(1, NUM_EPOCHS+1):
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        test_loss  = eval_epoch(model, test_loader,  criterion)
        print(f"Epoch {epoch}/{NUM_EPOCHS}  Train Loss: {train_loss:.4f}  Test Loss: {test_loss:.4f}")

    # Save model and scaler
    torch.save({'model_state_dict': model.state_dict(), 'scaler': scaler}, 'model_nowcast.pth')

if __name__ == "__main__":
    main()


Epoch 1/50  Train Loss: 0.0172  Test Loss: 0.0159
Epoch 2/50  Train Loss: 0.0163  Test Loss: 0.0155
Epoch 3/50  Train Loss: 0.0161  Test Loss: 0.0155


KeyboardInterrupt: 